<a href="https://colab.research.google.com/github/urvashi2004/PlantDoctor/blob/main/Colab/CheckDisease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing Libraries and Mounting Google Drive
from google.colab import drive
import os
import splitfolders

# Mount Google Drive
drive.mount('/content/drive')

# Set the working directory
os.chdir('/content/drive/My Drive/Colab Notebooks/PlantDoctor')

# Dataset Directory
base_dir = 'PlantVillageDS'

# Verify if the directory exists
if not os.path.exists(base_dir):
    raise ValueError(f"The provided input folder '{base_dir}' does not exist.")
else:
    print(f"Directory exists: {os.path.abspath(base_dir)}")

# Split dataset into train, validation, and test sets (if not already split)
output_dir = 'output_split'  # Output directory for split data
if not os.path.exists(output_dir):
    splitfolders.ratio(base_dir, output=output_dir, seed=42, ratio=(.8, .1, .1))
    print(f"Dataset split into: {output_dir}")
else:
    print(f"Split dataset already exists in: {output_dir}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directory exists: /content/drive/My Drive/Colab Notebooks/PlantDoctor/PlantVillageDS


Copying files: 18507 files [20:29, 55.45 files/s]

In [ ]:
# Import Libraries for Model Training
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

# Set paths for training, validation, and test sets
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Define the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Save the model
model.save('PlantDiseaseModel.h5')

Found 16504 images belonging to 15 classes.
Found 2058 images belonging to 15 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


516/516 ━━━━━━━━━━━━━━━━━━━━ 1130s 2s/step - accuracy: 0.4528 - loss: 1.6660 - val_accuracy: 0.7677 - val_loss: 0.6691
Epoch 2/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1039s 2s/step - accuracy: 0.8167 - loss: 0.5362 - val_accuracy: 0.8474 - val_loss: 0.4296
Epoch 3/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1018s 2s/step - accuracy: 0.8882 - loss: 0.3275 - val_accuracy: 0.8707 - val_loss: 0.3911
Epoch 4/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1032s 2s/step - accuracy: 0.9281 - loss: 0.2147 - val_accuracy: 0.8805 - val_loss: 0.3567
Epoch 5/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1001s 2s/step - accuracy: 0.9564 - loss: 0.1373 - val_accuracy: 0.9237 - val_loss: 0.2318
Epoch 6/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1044s 2s/step - accuracy: 0.9704 - loss: 0.0927 - val_accuracy: 0.9018 - val_loss: 0.3498
Epoch 7/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1059s 2s/step - accuracy: 0.9675 - loss: 0.0920 - val_accuracy: 0.9237 - val_loss: 0.2645
Epoch 8/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1002s 2s/step - accuracy: 0.9799 - loss: 0.0597 - val_accurac

In [ ]:
# Import Libraries for Evaluation
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

import os
model_path = '/content/drive/My Drive/Colab Notebooks/PlantDoctor/PlantDiseaseModel.h5'


# Load the saved model (if you are running this separately)
model = tf.keras.models.load_model(model_path)

# Evaluate the model on the test set
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc * 100:.2f}%")

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: 'output_split/test'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files
import os

# Function to manually upload the image
def upload_image():
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')
        return filename

# Function to preprocess the image and predict health status
def predict_plant_health(image_path, model):
    # Load and preprocess the image
    img = tf.keras.utils.load_img(image_path, target_size=(150, 150))  # Adjust target_size based on your model input size
    img_array = tf.keras.utils.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize the image

    # Make the prediction
    prediction = model.predict(img_array)

    # If the model is binary (single output neuron)
    if prediction.shape[-1] == 1:
        # Assuming the model outputs a value close to 1 for 'healthy' and 0 for 'diseased'
        if prediction[0][0] > 0.5:
            return "Healthy Plant"
        else:
            return "Diseased Plant"
    else:
        # If the model is multi-class (multiple output neurons)
        class_index = np.argmax(prediction)
        # Map class_index to the corresponding label
        class_labels = ["Tomato_Bacterial_spot", "Tomato_Early_blight", "Tomato_Late_blight", "Tomato_Leaf_Mold", "Tomato_Septoria_leaf_spot", "Tomato_Spider_mites_Two_spotted_spider_mite", "Tomato_Target_Spot", "Tomato__Tomato_YellowLeaf__Curl_Virus", "Tomato__Tomato_mosaic_virus", "Tomato_healthy", "Potato___Early_blight", "Potato___Late_blight", "Potato___healthy", "Pepper__bell___Bacterial_spot", "Pepper__bell___healthy"]
        return class_labels[class_index]

# Load the trained model
model_path = '/content/drive/My Drive/Colab Notebooks/PlantDoctor/PlantDiseaseModel.h5'
model = tf.keras.models.load_model(model_path, compile=False)

# Manually upload an image and predict
image_filename = upload_image()

# Check if the image exists and predict
if os.path.exists(image_filename):
    health_status = predict_plant_health(image_filename, model)
    print(f"Prediction: {health_status}")
else:
    print(f"File not found: {image_filename}")

Saving 0bfd3a50-768e-4661-a134-43aaee7a1c1d___Com.G_SpM_FL 8837.JPG to 0bfd3a50-768e-4661-a134-43aaee7a1c1d___Com.G_SpM_FL 8837.JPG
User uploaded file "0bfd3a50-768e-4661-a134-43aaee7a1c1d___Com.G_SpM_FL 8837.JPG" with length 21889 bytes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Prediction: Potato___Early_blight
